This python code has been developped by [Taïs Grippa](https://github.com/tgrippa) (Université Libre de Bruxelles). 

Code developped on Ubuntu 22.04 (Ubuntu Jammy) and GRASS GIS 8.0.2 using the Docker environment [available here](https://github.com/tgrippa/Weaksupervision_Vaihingen).

# Define working environment

**Import libraries**

In [1]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import glob
import math
import pickle
import time 
print(sys.version)

3.10.4 (main, Jun 29 2022, 12:14:53) [GCC 11.2.0]


In [2]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

**Add folder with SCR provided belong to this notebook**

In [3]:
# Add local module to the path
src = os.path.abspath('/home/tais/github/SRC')
if src not in sys.path:
    sys.path.append(src)

**Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev)**

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [4]:
run /home/tais/github/SRC/config.py

In [5]:
print(config_parameters)

{'GISBASE': '/usr/lib/grass78', 'PYTHONLIB': '/usr/bin/python3', 'gisdb': '/home/tais/GRASSDATA', 'location': 'flair-one', 'permanent_mapset': 'PERMANENT', 'locationepsg': '2154', 'outputfolder': '/home/tais/result', 'inputdir': '/home/tais/data'}


In [7]:
print(data)

{'legend': '/home/tais/github/Legend.txt'}


In [8]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [9]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

PATH	= /.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/grass78/bin:/usr/lib/grass78/script:/usr/lib/grass78/lib 	
HOSTNAME	= f6e49c08eb5b 	
DISPLAY	= unixlocalhost:10.0 	
LANG	= C.UTF-8 	
LC_ALL	= C.UTF-8 	
JUPYTER_ENABLE_LAB	= yes 	
TINI_VERSION	= v0.6.0 	
HOME	= /home/tais 	
GIT_PYTHON_REFRESH	= quiet 	
JPY_PARENT_PID	= 7 	
TERM	= xterm-color 	
CLICOLOR	= 1 	
PAGER	= cat 	
GIT_PAGER	= cat 	
MPLBACKEND	= module://matplotlib_inline.backend_inline 	
PYTHONPATH	= :/usr/lib/grass78/etc/python:/usr/lib/grass78/etc/python/grass:/usr/lib/grass78/etc/python/grass/script 	
LD_LIBRARY_PATH	= :/usr/lib/grass78/lib 	
GISBASE	= /usr/lib/grass78 	
PYTHONLIB	= /usr/bin/python3 	
GIS_LOCK	= $$ 	
GISRC	= /home/tais/.grass7/rc 	


**GRASS GIS Python libraries**

In [10]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

**Other functions**

In [11]:
from grass.script import vector
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name
# Import function that check and create folder
from mkdir import check_create_dir
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon
# Import function for sorting string number naturally
from sorting_natural import natural_keys

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

In [12]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Init
    gsetup.init(config_parameters['GISBASE'], config_parameters['gisdb'], config_parameters['location'], mapset)
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Return
    return return_message

In [13]:
def segmentation(image_id):
    # Define computational region
    gscript.run_command('g.region', overwrite=True, raster='red', save="region")
    # Define group of layers
    gscript.run_command('g.remove', flags='f', type='group', name='top')
    gscript.run_command('i.group', group='top', input="nir,red,green,blue")
    # gscript.run_command('i.group', group='top', input="nir,red,green,blue,ndsm") # TODO: Test of nDSM in the segmentation  
    # Unsupervised Segmentation parameter optimization 
    gscript.run_command('i.segment.uspo', overwrite=True, group='top', segment_map='best_segment', regions='region', segmentation_method='region_growing', 
                        threshold_start='0.005', threshold_stop='0.05', threshold_step='0.002', minsizes='50', memory='3000', processes='25')
    # Trick the segment ID to be sure the ID is unique through all the images
    formula = "segmentation = (%s*1000000) + best_segment_region_rank1"%image_id
    gscript.mapcalc(formula, overwrite=True)
    # Exportation
    gscript.run_command('r.out.gdal', overwrite=True, flags='m', input='segmentation', output='/home/tais/result/segment_rast/segment_%s.tiff'%image_id, format='GTiff')

In [14]:
def compute_stats(image_id):
    # Define computational region
    gscript.run_command('g.region', overwrite=True, raster='segmentation')
    # Define list of raster on which to compute statistics
    rast_layers=[]
    rast_layers.append('TOP.nir')
    rast_layers.append('TOP.red')
    rast_layers.append('TOP.green')
    rast_layers.append('text_green_DE')
    rast_layers.append('text_green_Entr')
    rast_layers.append('text_red_ASM')
    rast_layers.append('text_red_IDM')
    rast_layers.append('text_nir_DE')
    rast_layers.append('text_dsm_Entr')
    rast_layers.append('text_dsm_DE')
    rast_layers.append('ndvi')
    # Compute segment statistics
    gscript.run_command('i.segment.stats', overwrite=True, map='segmentation',
                        rasters=','.join(rast_layers), raster_statistics='stddev,coeff_var,sum,median,perc_90', area_measures='area,perimeter,compact_circle,compact_square,fd',
                        vectormap='segment', processes='20')   
    # Compute mode of GTS (Ground truth label)
    if image_id in config_parameters['images_val'] or image_id in config_parameters['images_classical_approach']:
        gscript.run_command('r.zonal.classes', overwrite=True, zone_map='segmentation',
                            raster='gts', statistics='mode', csvfile='/home/tais/result/stats/tmp_statszonal_%s.csv'%image_id, separator='comma')
        gscript.run_command('db.in.ogr', overwrite=True, input='/home/tais/result/stats/tmp_statszonal_%s.csv'%image_id, output='tmp_table')
        gscript.run_command('v.db.join', map='segment', column='cat', other_table='tmp_table', other_column='cat_', subset_columns='mode')
        gscript.run_command('v.db.renamecolumn', map='segment', column='mode,gts_label')
        os.remove('/home/tais/result/stats/tmp_statszonal_%s.csv'%image_id)
    # Export vector layer as GPKG
    gscript.run_command('v.out.ogr', overwrite=True, input='segment', output='/home/tais/result/segment_vect/segment_%s.gpkg'%image_id, format='GPKG')
    # Export attribute table as CSV
    gscript.run_command('v.db.select', overwrite=True, map='segment', separator='comma', file='/home/tais/result/stats/stats_%s.csv'%image_id)

In [16]:
def worker(job_tupple):
    # Launch a new mapset for this image
    launch_mapset(job_tupple[0])
    # Segmentation
    segmentation(image_id)
    #segmentation_with_slic(image_id)
    # Compute statistics
    try:
        compute_stats(image_id)
    except:   # Sometimes, the execution failed and the grass gis mapset need to be 'cleaned' by opening and closing it with the following command
        subprocess.check_call(['grass','--text','/home/tais/GRASSDATA/vaihingen/image_%s'%x])
        compute_stats(image_id)

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Create new directories

In [15]:
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder'])
check_create_dir(os.path.join(config_parameters['outputfolder'],'stats'))
check_create_dir(os.path.join(config_parameters['outputfolder'],'segment_rast'))
check_create_dir(os.path.join(config_parameters['outputfolder'],'segment_vect'))

The folder '/home/tais/result' already exists
The folder '/home/tais/result/stats' has been created
The folder '/home/tais/result/segment_rast' has been created
The folder '/home/tais/result/segment_vect' has been created


# Segmentation (region growing)

In [13]:
import fnmatch

def find_files(path, pattern):
    for root, dirs, files in os.walk(path):
        for file in fnmatch.filter(files, pattern):
            yield os.path.join(root, file)

In [14]:
# Get a list of path to train images
train_folder = os.path.join(config_parameters['inputdir'],'train')
train_tupple = [(os.path.split(x)[-1].split('.tif')[0],x) for x in find_files(train_folder, "IMG_*.tif")]
print(f'There are {len(train_tupple)} train images')

There are 2950 train images
